In [ ]:
%pip install -qU langchain-openai langchain langgraph langchain_community langgraph-supervisor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 473.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 5.5 MB/s eta 0:00:00


In [ ]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain_core.tools import tool
from langchain_core.messages import (
HumanMessage,
)
from langgraph.prebuilt import create_react_agent
import requests
key = userdata.get('OPENAI_API_KEY')

In [ ]:
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

In [ ]:
import requests

url = "https://spotify-scraper.p.rapidapi.com/v1/search"

querystring = {"term":"sleep music","type":"playlist","limit":"10"}

headers = {
	"x-rapidapi-key": "api key here",
	"x-rapidapi-host": "spotify-scraper.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'status': False, 'message': 'Config Error!2 Please try again.'}


In [ ]:
from langchain.tools import tool
import requests

@tool
def search_spotify_playlist(query: str) -> str:
    """
    Search for a Spotify playlist by query using the RapidAPI Spotify Scraper API.
    Returns the top playlist result with its name, share URL, owner, and description.
    """
    url = "https://spotify-scraper.p.rapidapi.com/v1/search"
    params = {
        "term": query,
        "type": "playlist",
        "limit": "10"
    }
    headers = {
        "x-rapidapi-key": "api key here",  # Replace with your actual API key
        "x-rapidapi-host": "spotify-scraper.p.rapidapi.com"
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()

        playlists = data.get("playlists", {}).get("items", [])
        if not playlists:
            return "No playlists found for your query."

        top_playlist = playlists[0]
        name = top_playlist.get("name", "Unknown Title")
        share_url = top_playlist.get("shareUrl", "No URL available")
        description = top_playlist.get("description", "No description available.")
        owner_info = top_playlist.get("owner", {})
        owner_name = owner_info.get("name", "Unknown owner")

        return (
            f"🎵 Playlist: {name}\n"
            f"👤 Created by: {owner_name}\n"
            f"📝 Description: {description}\n"
            f"🔗 Link: {share_url}"
        )

    except requests.exceptions.RequestException as e:
        return f"Error during Spotify search: {str(e)}"
    except (KeyError, IndexError, TypeError) as e:
        return f"Unexpected response format: {str(e)}"


In [ ]:
spotify_assistant = create_react_agent(
    model=model,
    tools=[search_spotify_playlist],
    prompt="You are a helpful Spotify assistant. Your job is to help users discover Spotify playlists based on their search queries. ",
    name="spotify_assistant"
)


In [ ]:
response = spotify_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="I can't sleep well.")
            ]
    }
    )
response["messages"]

[HumanMessage(content="I can't sleep well.", additional_kwargs={}, response_metadata={}, id='4905abf7-af31-4c5f-a1f9-d3174e380f98'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_aR9LAVV5TuAFJmuxsfYQX4NV', 'function': {'arguments': '{"query":"sleep"}', 'name': 'search_spotify_playlist'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 100, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BghnvoZasHtnUlYHhljJOGMRtFLyp', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='spotify_assistant', id='run--b1b5a4cc-4bff-43a8-8146-d3862f4ccb3f-0', tool_calls=[{'name': 'search_spotify_playlist', 'args': {'

In [ ]:
response["messages"][-1].content

'I found a great playlist that might help you sleep better:\n\n### [Fall into Sleep Instantly](https://open.spotify.com/playlist/6L0keGopfWZMQ76K4O8bX1)\n- **Created by:** J\n- **Description:** Perfect for helping you drift off into a deep sleep.\n\nGive it a listen and see if it helps you relax!'

In [ ]:
response = spotify_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="cant sleep well.")
            ]
    }
    )
response["messages"]

[HumanMessage(content='cant sleep well.', additional_kwargs={}, response_metadata={}, id='9f3128f4-4b4b-4047-b5c5-c067244b8a28'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3NHErevK5z3c5AFaqQOjcAtW', 'function': {'arguments': '{"query":"sleep music"}', 'name': 'search_spotify_playlist'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 99, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bghpx78LEJ7EQ4tfh46Cu89lfjDBJ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='spotify_assistant', id='run--44a4a015-e1f9-45a5-912d-dde29ffe5237-0', tool_calls=[{'name': 'search_spotify_playlist', 'args': 

In [ ]:
response["messages"][-1].content

"I found a great playlist for you! \n\n**Playlist:** [Sleep](https://open.spotify.com/playlist/37i9dQZF1DWZd79rJ6a7lp)  \n**Created by:** Spotify  \n**Description:** Gentle Ambient piano to help you fall asleep.  \n\nGive it a listen and hopefully, it will help you get a better night's sleep!"

### Response obtained in rapidapi but all not being retrieved

In [ ]:
{
  "status": true,
  "errorId": "Success",
  "playlists": {
    "totalCount": 862,
    "items": [
      {
        "type": "playlist",
        "id": "5RTuZuGxqBWYC4cHZNWYXR",
        "name": "POV you wanna cry but you can't so you hold it all in till you can..</3",
        "shareUrl": "https://open.spotify.com/playlist/5RTuZuGxqBWYC4cHZNWYXR",
        "description": "Are we all just mentally unstable here?Also, Tysm for 1000+ saves! ily all&lt;3 Stay hydrated, eat, and sleep well!",
        "owner": {
          "type": "user",
          "id": "31uefoof6dsi2ct2lexqccfhlisu",
          "name": "April <3",
          "shareUrl": "https://open.spotify.com/user/31uefoof6dsi2ct2lexqccfhlisu",
          "avatar": [
            {
              "url": "https://i.scdn.co/image/ab67757000003b82116d84bfccc7aba0b6dae086",
              "width": 64,
              "height": 64
            },
            {
              "url": "https://i.scdn.co/image/ab6775700000ee85116d84bfccc7aba0b6dae086",
              "width": 300,
              "height": 300
            }
          ]
        },
        "images": [
          [
            {
              "url": "https://image-cdn-ak.spotifycdn.com/image/ab67706c0000d72c328705826ebc073722c3d0ab",
              "width": null,
              "height": null
            }
          ]
        ]
      },
      {
        "type": "playlist",
        "id": "2v4eaJiWTsIXO9tDPvQnxj",
        "name": "sleepy sleep sleep",
        "shareUrl": "https://open.spotify.com/playlist/2v4eaJiWTsIXO9tDPvQnxj",
        "description": "i think this playlist got hacked :&#x2F; did my best to fix it",
        "owner": {
          "type": "user",
          "id": "flowerpower990",
          "name": "Gemma Laine <3",
          "shareUrl": "https://open.spotify.com/user/flowerpower990",
          "avatar": [
            {
              "url": "https://i.scdn.co/image/ab67757000003b826a041949412d0f34228bfd93",
              "width": 64,
              "height": 64
            },
            {
              "url": "https://i.scdn.co/image/ab6775700000ee856a041949412d0f34228bfd93",
              "width": 300,
              "height": 300
            }
          ]
        },
        "images": [
          [
            {
              "url": "https://mosaic.scdn.co/60/ab67616d00001e025f592cf31f9602e45c1ddf16ab67616d00001e026b701428ed4c6e053902174aab67616d00001e02820e2ac14772ae3162c6d479ab67616d00001e029b7190e673e46271b2754aab",
              "width": 60,
              "height": 60
            },
            {
              "url": "https://mosaic.scdn.co/300/ab67616d00001e025f592cf31f9602e45c1ddf16ab67616d00001e026b701428ed4c6e053902174aab67616d00001e02820e2ac14772ae3162c6d479ab67616d00001e029b7190e673e46271b2754aab",
              "width": 300,
              "height": 300
            },
            {
              "url": "https://mosaic.scdn.co/640/ab67616d00001e025f592cf31f9602e45c1ddf16ab67616d00001e026b701428ed4c6e053902174aab67616d00001e02820e2ac14772ae3162c6d479ab67616d00001e029b7190e673e46271b2754aab",
              "width": 640,
              "height": 640
            }
          ]
        ]
      },
      {
        "type": "playlist",
        "id": "167j9o9fVldFcC5Zzu9ou1",
        "name": "a playlist for when you can't sleep",
        "shareUrl": "https://open.spotify.com/playlist/167j9o9fVldFcC5Zzu9ou1",
        "description": "i hope you sleep well, sweet dreams ♡",
        "owner": {
          "type": "user",
          "id": "jphxmh4ie79kdgy403pm9n3cl",
          "name": "☆ 𝔑 ☆",
          "shareUrl": "https://open.spotify.com/user/jphxmh4ie79kdgy403pm9n3cl",
          "avatar": [
            {
              "url": "https://i.scdn.co/image/ab67757000003b82acaa136bac56c3090a0a01cd",
              "width": 64,
              "height": 64
            },
            {
              "url": "https://i.scdn.co/image/ab6775700000ee85acaa136bac56c3090a0a01cd",
              "width": 300,
              "height": 300
            }
          ]
        },
        "images": [
          [
            {
              "url": "https://image-cdn-fa.spotifycdn.com/image/ab67706c0000d72c3a127e2aae23608128cf5c0d",
              "width": null,
              "height": null
            }
          ]
        ]
      },
      {
        "type": "playlist",
        "id": "34jITz4o38pOONWiJkjVYs",
        "name": "Sleep Well 1 hour CG5",
        "shareUrl": "https://open.spotify.com/playlist/34jITz4o38pOONWiJkjVYs",
        "description": "",
        "owner": {
          "type": "user",
          "id": "31q2ghwoqs36rx3lkfei4osyvlwi",
          "name": "°•.Bestie-Rocky.•°",
          "shareUrl": "https://open.spotify.com/user/31q2ghwoqs36rx3lkfei4osyvlwi",
          "avatar": [
            {
              "url": "https://i.scdn.co/image/ab67757000003b82b766e83f1bc6af25e277a9d9",
              "width": 64,
              "height": 64
            },
            {
              "url": "https://i.scdn.co/image/ab6775700000ee85b766e83f1bc6af25e277a9d9",
              "width": 300,
              "height": 300
            }
          ]
        },
        "images": [
          [
            {
              "url": "https://image-cdn-fa.spotifycdn.com/image/ab67706c0000d72c05431141c7ec6bb826f2a0cf",
              "width": null,
              "height": null
            }
          ]
        ]
      },
      {
        "type": "playlist",
        "id": "2tdpr0BxQnq4trDANQRnRl",
        "name": "Sleep well love❤️",
        "shareUrl": "https://open.spotify.com/playlist/2tdpr0BxQnq4trDANQRnRl",
        "description": "Listen to when you miss me or need to sleep and can’t call me for a jam or anything of the sort. I like you so much I’ll tell you I love you soon. 😘❤️",
        "owner": {
          "type": "user",
          "id": "0povmh6o8uvsp6u7rgcd15yz5",
          "name": "BugBug",
          "shareUrl": "https://open.spotify.com/user/0povmh6o8uvsp6u7rgcd15yz5",
          "avatar": [
            {
              "url": "https://i.scdn.co/image/ab67757000003b8262c15a788abb1bb99f9dcd67",
              "width": 64,
              "height": 64
            },
            {
              "url": "https://i.scdn.co/image/ab6775700000ee8562c15a788abb1bb99f9dcd67",
              "width": 300,
              "height": 300
            }
          ]
        },
        "images": [
          [
            {
              "url": "https://image-cdn-ak.spotifycdn.com/image/ab67706c0000d72cc026f5d543c7a65129c8e457",
              "width": null,
              "height": null
            }
          ]
        ]
      },
      {
        "type": "playlist",
        "id": "3QthSp4HKzwcCluDQjV0E4",
        "name": "can't sleep at 4am😓",
        "shareUrl": "https://open.spotify.com/playlist/3QthSp4HKzwcCluDQjV0E4",
        "description": "tomorrow might be a better day goodnight loves&lt;3",
        "owner": {
          "type": "user",
          "id": "tfbmb109kv1lkdqldl1uzhe17",
          "name": "i s a b e l l e <3",
          "shareUrl": "https://open.spotify.com/user/tfbmb109kv1lkdqldl1uzhe17",
          "avatar": [
            {
              "url": "https://i.scdn.co/image/ab67757000003b822ab0313358977c36c111709f",
              "width": 64,
              "height": 64
            },
            {
              "url": "https://i.scdn.co/image/ab6775700000ee852ab0313358977c36c111709f",
              "width": 300,
              "height": 300
            }
          ]
        },
        "images": [
          [
            {
              "url": "https://mosaic.scdn.co/60/ab67616d00001e02121a9af58f3604f78dd68f6bab67616d00001e0213584782bb6d165483d296edab67616d00001e02343b0fd081b190473e8a9f5cab67616d00001e0260ba1d6104d0475c7555a6b2",
              "width": 60,
              "height": 60
            },
            {
              "url": "https://mosaic.scdn.co/300/ab67616d00001e02121a9af58f3604f78dd68f6bab67616d00001e0213584782bb6d165483d296edab67616d00001e02343b0fd081b190473e8a9f5cab67616d00001e0260ba1d6104d0475c7555a6b2",
              "width": 300,
              "height": 300
            },
            {
              "url": "https://mosaic.scdn.co/640/ab67616d00001e02121a9af58f3604f78dd68f6bab67616d00001e0213584782bb6d165483d296edab67616d00001e02343b0fd081b190473e8a9f5cab67616d00001e0260ba1d6104d0475c7555a6b2",
              "width": 640,
              "height": 640
            }
          ]
        ]
      },
      {
        "type": "playlist",
        "id": "4SqE7B2RwLMwHApieiw7NN",
        "name": "Looking for the good stuff???(basically music crack)",
        "shareUrl": "https://open.spotify.com/playlist/4SqE7B2RwLMwHApieiw7NN",
        "description": "Can’t sleep? Well then here is music that is basically melatonin (your either gonna love it or hate it)(indie)",
        "owner": {
          "type": "user",
          "id": "a0aqf41dgfhpj9oixtdbxlieo",
          "name": "Banana montana",
          "shareUrl": "https://open.spotify.com/user/a0aqf41dgfhpj9oixtdbxlieo",
          "avatar": [
            {
              "url": "https://i.scdn.co/image/ab67757000003b82f96cffc4a6671afe8088f299",
              "width": 64,
              "height": 64
            },
            {
              "url": "https://i.scdn.co/image/ab6775700000ee85f96cffc4a6671afe8088f299",
              "width": 300,
              "height": 300
            }
          ]
        },
        "images": [
          [
            {
              "url": "https://image-cdn-ak.spotifycdn.com/image/ab67706c0000d72c6ca550c1e69a73c8e33a0c85",
              "width": null,
              "height": null
            }
          ]
        ]
      },
      {
        "type": "playlist",
        "id": "4eZxQ61VWlwnYYLuIfN5hy",
        "name": "pov: you can't sleep at 2am",
        "shareUrl": "https://open.spotify.com/playlist/4eZxQ61VWlwnYYLuIfN5hy",
        "description": "lonely nights",
        "owner": {
          "type": "user",
          "id": "bkmehbgdwm0jbwftitsce9kry",
          "name": "Konstancja",
          "shareUrl": "https://open.spotify.com/user/bkmehbgdwm0jbwftitsce9kry",
          "avatar": [
            {
              "url": "https://i.scdn.co/image/ab67757000003b82aa9c887d4edc842647be3559",
              "width": 64,
              "height": 64
            },
            {
              "url": "https://i.scdn.co/image/ab6775700000ee85aa9c887d4edc842647be3559",
              "width": 300,
              "height": 300
            }
          ]
        },
        "images": [
          [
            {
              "url": "https://image-cdn-ak.spotifycdn.com/image/ab67706c0000d72ccf451644880f65593522098f",
              "width": null,
              "height": null
            }
          ]
        ]
      },
      {
        "type": "playlist",
        "id": "31xbK0DbPHGtePFAdT3Gk9",
        "name": "Kdrama OST ( calm & soothing )",
        "shareUrl": "https://open.spotify.com/playlist/31xbK0DbPHGtePFAdT3Gk9",
        "description": "For those who cant sleep, this is for u. Rest well. ♡",
        "owner": {
          "type": "user",
          "id": "hdywghlwyurs5rfvbp1d1kurc",
          "name": "huda0.fm",
          "shareUrl": "https://open.spotify.com/user/hdywghlwyurs5rfvbp1d1kurc",
          "avatar": [
            {
              "url": "https://i.scdn.co/image/ab67757000003b82f7e9129464b64efc4f6fa770",
              "width": 64,
              "height": 64
            },
            {
              "url": "https://i.scdn.co/image/ab6775700000ee85f7e9129464b64efc4f6fa770",
              "width": 300,
              "height": 300
            }
          ]
        },
        "images": [
          [
            {
              "url": "https://image-cdn-ak.spotifycdn.com/image/ab67706c0000d72c7625ec6087a7a8ece2712be7",
              "width": null,
              "height": null
            }
          ]
        ]
      },
      {
        "type": "playlist",
        "id": "4tBxB1JFYCMN6hSgfoxMQ5",
        "name": "for when you can’t sleep ",
        "shareUrl": "https://open.spotify.com/playlist/4tBxB1JFYCMN6hSgfoxMQ5",
        "description": "yes they&#x27;re in the exact order",
        "owner": {
          "type": "user",
          "id": "9ipx060ar88lobiqvwv9i92ci",
          "name": "Miriam Sink",
          "shareUrl": "https://open.spotify.com/user/9ipx060ar88lobiqvwv9i92ci",
          "avatar": [
            {
              "url": "https://i.scdn.co/image/ab67757000003b82c58964ccbf6de74e1d7562b2",
              "width": 64,
              "height": 64
            },
            {
              "url": "https://i.scdn.co/image/ab6775700000ee85c58964ccbf6de74e1d7562b2",
              "width": 300,
              "height": 300
            }
          ]
        },
        "images": [
          [
            {
              "url": "https://mosaic.scdn.co/60/ab67616d00001e020b1993db92eecc2dd5021293ab67616d00001e02a3e3d57bdadd06f26ea04e62ab67616d00001e02ceca3ef39ab8f9b59acff394ab67616d00001e02ebcc3ef64dbef8acb47e2960",
              "width": 60,
              "height": 60
            },
            {
              "url": "https://mosaic.scdn.co/300/ab67616d00001e020b1993db92eecc2dd5021293ab67616d00001e02a3e3d57bdadd06f26ea04e62ab67616d00001e02ceca3ef39ab8f9b59acff394ab67616d00001e02ebcc3ef64dbef8acb47e2960",
              "width": 300,
              "height": 300
            },
            {
              "url": "https://mosaic.scdn.co/640/ab67616d00001e020b1993db92eecc2dd5021293ab67616d00001e02a3e3d57bdadd06f26ea04e62ab67616d00001e02ceca3ef39ab8f9b59acff394ab67616d00001e02ebcc3ef64dbef8acb47e2960",
              "width": 640,
              "height": 640
            }
          ]
        ]
      }
    ]
  }
}

In [ ]:
import requests
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

class SpotifyAgent:
    def __init__(self, api_key=None, rapidapi_key=None):
        self.api_key = api_key or userdata.get('OPENAI_API_KEY')
        self.rapidapi_key = rapidapi_key or "api key here"  # Replace with your RapidAPI key
        self.model = ChatOpenAI(api_key=self.api_key, model="gpt-4o-mini")
        self.agent = create_react_agent(
            model=self.model,
            tools=[self.search_spotify_playlist],
            prompt="You are a helpful Spotify assistant. Your job is to help users discover Spotify playlists based on their search queries.",
            name="spotify_assistant"
        )

    @tool
    def search_spotify_playlist(self, query: str) -> str:
        """
        Search for a Spotify playlist by query using the RapidAPI Spotify Scraper API.
        Returns the top playlist result with its name, share URL, owner, and description.
        """
        url = "https://spotify-scraper.p.rapidapi.com/v1/search"
        params = {
            "term": query,
            "type": "playlist",
            "limit": "10"
        }
        headers = {
            "x-rapidapi-key": self.rapidapi_key,
            "x-rapidapi-host": "spotify-scraper.p.rapidapi.com"
        }
        try:
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()
            data = response.json()
            playlists = data.get("playlists", {}).get("items", [])
            if not playlists:
                return "No playlists found for your query."
            top_playlist = playlists[0]
            name = top_playlist.get("name", "Unknown Title")
            share_url = top_playlist.get("shareUrl", "No URL available")
            description = top_playlist.get("description", "No description available.")
            owner_info = top_playlist.get("owner", {})
            owner_name = owner_info.get("name", "Unknown owner")
            return (
                f"🎵 Playlist: {name}\n"
                f"👤 Created by: {owner_name}\n"
                f"📝 Description: {description}\n"
                f"🔗 Link: {share_url}"
            )
        except requests.exceptions.RequestException as e:
            return f"Error during Spotify search: {str(e)}"
        except (KeyError, IndexError, TypeError) as e:
            return f"Unexpected response format: {str(e)}"

    def run(self, user_input):
        response = self.agent.invoke({
            "messages": [HumanMessage(content=user_input)]
        })
        return response["messages"][-1].content

# Example usage:
spotify_agent = SpotifyAgent()
result = spotify_agent.run("relaxing sleep music playlist")
print(result)